In [1]:
# from langchain_ollama import ChatOllama

# print("Test with tag…")
# try:
#     llm = ChatOllama(model="llama3.2:latest", base_url="http://127.0.0.1:11434", temperature=0)
#     print(llm.invoke("Say hi (tag).").content)
# except Exception as e:
#     print("Tag failed:", e)

# print("Test without tag…")
# try:
#     llm = ChatOllama(model="llama3.2", base_url="http://127.0.0.1:11434", temperature=0)
#     print(llm.invoke("Say hi (no tag).").content)
# except Exception as e:
#     print("No-tag failed:", e)


## Deep Agents 0.2 

In [ ]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional

# =============================
# 0. SETUP: Mocks / Utilities
# =============================

@dataclass
class Message:
    role: str
    content: str

@dataclass
class MockAgent:
    name: str
    memory: Dict[str, Any] = field(default_factory=dict)

    def _get_thread(self, config: Optional[Dict[str, Any]] = None) -> str:
        cfg = (config or {}).get("configurable", {})
        return cfg.get("thread_id", "default-thread")

    def invoke(self, payload: Dict[str, Any], config: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        thread = self._get_thread(config)
        msgs: List[Message] = [Message(**m) if not isinstance(m, Message) else m for m in payload.get("messages", [])]

        # Super simple behavior & memory
        last_user = next((m for m in reversed(msgs) if m.role == "user"), None)

        # Memory store/recall
        if last_user and "favorite topic is" in last_user.content.lower():
            topic = last_user.content.split("is", 1)[-1].strip().rstrip(".")
            self.memory.setdefault(thread, {})["favorite_topic"] = topic
            reply = f"Noted. I'll remember your favorite topic is {topic}."
        elif last_user and "what's my favorite topic" in last_user.content.lower():
            topic = self.memory.get(thread, {}).get("favorite_topic", "unknown")
            reply = f"You told me your favorite topic is {topic}."
        elif last_user and "plan:" in last_user.content.lower():
            # Make a tiny plan
            steps = ["Research requirement", "Draft outline", "Implement", "Test", "Deliver"]
            reply = "Plan generated:\n" + "\n".join(f"{i+1}. {s}" for i, s in enumerate(steps))
        elif last_user and "search(" in last_user.content.lower():
            # mock search tool usage
            reply = "Search results: ['doc_a', 'doc_b', 'doc_c']"
        else:
            reply = f"Hello from {self.name}. You said: {last_user.content if last_user else '(no input)'}"

        result_msgs = msgs + [Message(role="assistant", content=reply)]
        return {
            "messages": [m.__dict__ for m in result_msgs],
            "state": {"thread_id": thread, "memory": self.memory.get(thread, {})}
        }

# Subagent container for demonstration
@dataclass
class MockCoordinator:
    planner: MockAgent
    researcher: MockAgent
    implementer: MockAgent

    def invoke(self, payload: Dict[str, Any], config: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        # very naive: planner -> researcher -> implementer
        thread = (config or {}).get("configurable", {}).get("thread_id", "default-thread")
        messages = payload.get("messages", [])

        p = self.planner.invoke({"messages": messages + [Message(role="user", content="Plan: build a demo").__dict__]}, config)
        r = self.researcher.invoke({"messages": p["messages"] + [Message(role="user", content="search(query='demo')").__dict__]}, config)
        i = self.implementer.invoke({"messages": r["messages"] + [Message(role="user", content="Implement based on findings.").__dict__]}, config)

        return i

In [3]:
# ============================================
# 1. SETUP: Define a simple tool
# ============================================

def mock_search(query: str, max_results: int = 3) -> str:
    """Mock search tool that returns dummy results"""
    return f"Search results for '{query}': Found {max_results} relevant articles about {query}."

In [4]:
# ============================================
# 2. CREATE AGENT: Test basic functionality
# ============================================

def test_basic_agent():
    """Test 1: Basic agent creation with default backend"""
    print("\n=== Test 1: Basic Deep Agent ===")
    
    agent = create_deep_agent(
        model=make_local_model(),
        tools=[mock_search],
        system_prompt="""You are a research assistant. 
        Use the search tool to find information and write results to files."""
    )
    
    # Invoke with a simple task
    result = agent.invoke({
        "messages": [{"role": "user", "content": "Search for 'LangChain agents' and create a summary file."}]
    })
    
    print(f"✓ Agent created and invoked successfully")
    print(f"✓ Total messages: {len(result['messages'])}")
    return agent

In [5]:
# ============================================
# 3. TEST SUBAGENTS: Validate subagent spawning
# ============================================

def test_subagents():
    """Test 2: Agent with specialized subagent"""
    print("\n=== Test 2: Agent with Subagents ===")
    
    research_subagent = {
        "name": "deep-researcher",
        "description": "Specialized agent for in-depth research tasks",
        "system_prompt": "You are an expert researcher who provides detailed analysis.",
        "tools": [mock_search],
        "model": make_local_model(),
    }
    
    agent = create_deep_agent(
        model=make_local_model(),
        system_prompt="You are a supervisor. Delegate research tasks to the deep-researcher subagent.",
        subagents=[research_subagent]
    )
    
    result = agent.invoke({
        "messages": [{"role": "user", "content": "Research LangGraph and provide a detailed report."}]
    })
    
    print(f"✓ Subagent configuration successful")
    print(f"✓ Total messages: {len(result['messages'])}")
    return agent

In [6]:
# ============================================
# 4. TEST BACKEND: Validate pluggable backend
# ============================================

def test_custom_backend():
    """Test 3: Agent with custom backend (new in 0.2)"""
    print("\n=== Test 3: Pluggable Backend (0.2 Feature) ===")
    
    try:
        from deepagents.backends.filesystem import FilesystemBackend
        
        backend = FilesystemBackend(root_dir="./agent_workspace", virtual_mode=False)
        agent = create_deep_agent(
            model=make_local_model(),
            tools=[mock_search],
            system_prompt="You are a file management assistant.",
            backend=backend,
        )
        
        print("✓ Custom backend configured successfully")
        return agent
    except ImportError as e:
        print(f"⚠ Backend test skipped: {e}")
        return None

In [7]:
# ============================================
# 5. TEST PLANNING: Validate todo list tool
# ============================================

def test_planning_tool():
    """Test 4: Validate built‑in planning capabilities"""
    print("\n=== Test 4: Planning Tool ===")

    planning_subagent = {
        "name": "research-analyst",
        "description": "Analyzes complex requests and drafts concise plans.",
        "system_prompt": (
            "Break the user goal into short, numbered action steps and return a bullet list."
        ),
        "tools": [mock_search],
        "model": make_local_model(),
    }

    agent = create_deep_agent(
        model=make_local_model(),
        tools=[mock_search],
        system_prompt="You must use the write_todos tool before starting complex tasks.",
        subagents=[planning_subagent],
    )

    result = agent.invoke({
        "messages": [{
            "role": "user",
            "content": "Create a multi-step plan to research and write a report on AI agents."
        }]
    })

    print("✓ Planning tool available")
    print(f"✓ Total messages: {len(result['messages'])}")
    return agent

In [8]:
# ============================================
# 6. TEST MEMORY: Validate checkpointing
# ============================================

def test_memory_persistence():
    """Test 5: Agent with memory/checkpointing"""
    print("\n=== Test 5: Memory Persistence ===")
    
    checkpointer = MemorySaver()
    
    agent = create_deep_agent(
        model=make_local_model(),
        tools=[mock_search],
        system_prompt="Remember previous conversations.",
        checkpointer=checkpointer
    )
    
    config = {"configurable": {"thread_id": "test-thread-1"}}
    
    # First interaction
    result1 = agent.invoke({
        "messages": [{"role": "user", "content": "My favorite topic is AI agents."}]
    }, config)
    
    # Second interaction (should remember context)
    result2 = agent.invoke({
        "messages": [{"role": "user", "content": "What's my favorite topic?"}]
    }, config)
    
    print(f"✓ Memory persistence configured")
    print(f"✓ Thread maintained across invocations")
    return agent

In [9]:
# ============================================
# 7. RUN ALL TESTS
# ============================================

def run_validation():
    """Run all validation tests"""
    print("\n" + "="*60)
    print("🧠 Deep Agents 0.2 Validation Suite")
    print("="*60)
    
    try:
        test_basic_agent()
        test_subagents()
        test_custom_backend()
        test_planning_tool()
        test_memory_persistence()
        
        print("\n" + "="*60)
        print("✅ All validation tests completed successfully!")
        print("="*60)
        
    except Exception as e:
        print(f"\n❌ Validation failed: {str(e)}")
        raise

if __name__ == "__main__":
    # Run validation suite
    run_validation()


🧠 Deep Agents 0.2 Validation Suite

=== Test 1: Basic Deep Agent ===
✓ Agent created and invoked successfully
✓ Total messages: 4

=== Test 2: Agent with Subagents ===
✓ Subagent configuration successful
✓ Total messages: 2

=== Test 3: Pluggable Backend (0.2 Feature) ===
✓ Custom backend configured successfully

=== Test 4: Planning Tool ===
✓ Planning tool available
✓ Total messages: 2

=== Test 5: Memory Persistence ===
✓ Memory persistence configured
✓ Thread maintained across invocations

✅ All validation tests completed successfully!
